In [1]:
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
import scipy.io, cv2

# Find images path
images_paths = glob('../Dataset_Final/jpg_images/*/*')

# Fulfill data
X_ = []
y_ = []
for path in images_paths:
    scene = path.split('/')[-2]
    
    # Input data (images)
    img = plt.imread(path)
    # Image resize
    if 1:
        img = cv2.resize(img, dsize = None,fx=0.6, fy=0.6, interpolation=cv2.INTER_CUBIC)
    
    X_.append(img)
    
    # Output data (NLFs)
    nlf_path = '../Dataset_Final/NLFs/NLF_' + scene + '.mat'
    mat = scipy.io.loadmat('../Dataset_Final/NLFs/NLF_' + scene + '.mat')
    NLF = mat["NLF"][0]
    y_.append(NLF)
    
#(X_train, y_train), (X_test, y_test) = ''

In [2]:
X_ = np.asarray(X_)
y_ = np.asarray(y_)



### Crear modelo

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, AveragePooling2D
from SpatialPyramidPooling import SpatialPyramidPooling

# create model (a)
model_a = Sequential()

# add model a layers
model_a.add(Conv2D(64, kernel_size=5, strides = 2, activation='relu', input_shape=(None,None,3))) # conv1_a
model_a.add(AveragePooling2D(pool_size=(2, 2))) # pool1_a
model_a.add(Conv2D(64, kernel_size=3, activation='relu')) # conv2_a
model_a.add(AveragePooling2D(pool_size=(2, 2))) # pool2_a
model_a.add(Conv2D(64, kernel_size=3, activation='relu')) # conv3_a
model_a.add(Conv2D(64, kernel_size=3, activation='relu')) # conv4_a

# spp
model_a.add(SpatialPyramidPooling([1, 2, 4]))

# fully connected layers
model_a.add(Dense(2048, activation='relu'))
model_a.add(Dense(256*3, activation='relu'))

# model_a.output

In [9]:
from keras import optimizers

# compile model (a)
sgd = optimizers.SGD(lr=0.001, decay=0.0001, momentum=0.9, nesterov=True)
model_a.compile(optimizer= sgd, loss='mse')

In [ ]:
model_a.fit(x=X_, y=y_, batch_size=32, epochs=10, validation_split=0.1, shuffle=True)


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 294 samples, validate on 33 samples
Epoch 1/10


### Tutoriales

In [11]:
import numpy as np
x = np.ones([2,2])
xx = np.ones([3,3])

z = []
z.append(x)
z.append(xx)
z = np.asarray(z)
print(type(z[0]))


<class 'numpy.ndarray'>


In [1]:
import scipy.io
mat = scipy.io.loadmat('../Dataset_Final/NLFs/NLF_' + 'Scene_05' + '.mat')
NLF = mat["NLF"][0]
print(type(NLF))


<class 'numpy.ndarray'>
